# VI: Second Practical Work

**Authors:** Gerard Comas & Marc Franquesa.

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd
import warnings
import altair_viewer

alt.data_transformers.disable_max_rows()
warnings.simplefilter(action="ignore", category=FutureWarning)

collisions = pd.read_csv("./processed-data/collisions_weather.csv")
map_data = gpd.read_file("./processed-data/map.geojson")


primary = "green"
secondary = ""

colors = {
    primary: "#66c2a4",
    secondary: "",
}

collisions["VALID"] = collisions["VALID"].astype("int64")

collisions.head()

La nostre idea per a respondre a totes les preguntes d'aquest segon projecte es crear un Dashboard interactiu que tindrà la distribució següent:
- En la primera fila tres barplots en el que es representen el total de colisions amb un filtre concret. Aquests tres barplots seràn independents. Els tres barplots compartiran l'escala i l'escala de colors per tal de poder apreciar millor les magnituds. 
    - El primer de tots serà un barplot amb el total de colisions per cada mes, amb la funcionalitat de poder seleccionat una de les barres i els gràfics de les altre files dependeràn d'aquest filtrat. Si no es selecciona cap es mostraran tots els mesos. 
    - El segon serà un barplot amb el total de colisions per cada tipus de vehicle, amb la mateixa funcionalitat que abans.
    - El tercer i últim serà un barplot amb el total de colisions per a cada condició climàtica, amb la mateia funcionalitat que abans. 
- En la segona fila trobarem un element:
    - Un mapa de New York, on primerament es mostrin les fronteres dels barris. Aquest mapa dera un chloropleth i l'intensitat del color indicarà el total de colisions pel filtrat seleccionat. 
        - Hem de decidir si l'escala serà independent o no del total
        - També, podriem afegir més granularitat temporal amb un desplegable per a canviar.
        - El mapa incorporarà un tooltip que ens identifiqui la localitació i també el nombre de colisions
        - A més, serà interactiu i pemetrà filtrar per als gràfics de les lines inferiors.
- En la tercera fila trobarem dos elements:
    - Un heatmap on les files indiquen les setmanes i a les columnes els dies de la setmana. Està dissenyat per a que recordi a un calendari. Quan no s'hagi seleccionat el mes es mostrarà unes opcions per a escollir el mes a representar. El color indicarà la quantitat de colisions aquell dia.
    - En el segon element trobarem un line chart, aquest mostrarà totes les línes per a cada area dels mesos seleccionats, si s'ha seleccionat un mes, només es mostrarnan la d'aquell mes i si s'ha seleccionat una area es resaltarà aquella linea per sobre les altres. A més, hi haurà un slider per a marcar quina hora volem analitar, indicant-nos amb punts sobre les lines els valors per a cada area i per tant poder obtenir els valors més facilment. 

Pot ser confús per l'usuari doncs no queda del tot clar quines gràfiques filtren i quines no igual que la dependencia entre gràfics. 

In [ ]:
month_order = ["June", "July", "August", "September"]
month_selection = alt.selection_point(fields=["MONTH"], empty=True)

vehicle_order = ["Taxi", "Ambulance", "Fire truck"]
vehicle_selection = alt.selection_point(fields=["VEHICLE"], empty=True)

weather_order = ["rain", "clear-day", "cloudy","partly-cloudy-day"]
weather_selection = alt.selection_point(fields=["WEATHER"], empty=True)

bars_df = collisions.groupby(["MONTH", "VEHICLE", "VEHICLE EMOJI", "WEATHER", "WEATHER EMOJI"]).agg({"VALID": "sum"}).reset_index()
months = (
    alt.Chart(bars_df)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("MONTH:O", sort=month_order, axis=alt.Axis(title="Month", labelAngle=0)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(month_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        month_selection
    ).transform_filter(
        vehicle_selection & weather_selection
    )
    .properties(width=250, height=175)
)

vehicles = (
    alt.Chart(bars_df)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("VEHICLE:N", sort=vehicle_order, axis=None),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(vehicle_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("VEHICLE:N", title="Vehicle"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        vehicle_selection
    ).transform_filter(
        month_selection & weather_selection
    )
    .properties(width=200, height=175)
)

vehicles += (
    alt.Chart(bars_df)
    .mark_text(size=18, align="center", dy=-8)
    .encode(
        x=alt.X("VEHICLE:N"),
        y=alt.Y("sum(VALID):Q"),
        text=alt.Text("VEHICLE EMOJI:N"),
        opacity=alt.condition(vehicle_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("VEHICLE:N", title="Vehicle"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        vehicle_selection
    ).transform_filter(
        month_selection & weather_selection
    )
)

weather = (
    alt.Chart(bars_df)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("WEATHER:N", sort=vehicle_order, axis=None),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(weather_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("WEATHER:N", title="Weather"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        weather_selection
    ).transform_filter(
        month_selection & vehicle_selection
    ).properties(width=200, height=175)
)

weather += (
    alt.Chart(bars_df)
    .mark_text(size=18, align="center", dy=-8)
    .encode(
        x=alt.X("WEATHER:N"),
        y=alt.Y("sum(VALID):Q"),
        text=alt.Text("WEATHER EMOJI:N"),
        opacity=alt.condition(weather_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("WEATHER:N", title="Weather"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        weather_selection
    ).transform_filter(
        month_selection & vehicle_selection
    )
)

In [ ]:
(months & (vehicles | weather))

Now we will add an interactive map of NY

In [ ]:
map_data["BOROUGH"] = map_data["boro_name"]
map_data["AREA_KM2"] = map_data["AREA KM2"]

In [ ]:
map_data.head()

In [ ]:
ny_map_selection = alt.selection_point(fields=["BOROUGH"], empty=True)

collisions_borough=collisions.groupby(["MONTH","VEHICLE","WEATHER","BOROUGH"]).agg({"VALID": "sum"}).reset_index()
map_data = map_data[["BOROUGH", "AREA_KM2", "geometry"]]

ny_map = (
    alt.Chart(collisions_borough)
    .mark_geoshape(stroke="gray")
    .project(type="albersUsa")
    .transform_lookup(
        lookup = "BOROUGH",                 
        from_ = alt.LookupData(data=map_data, key="BOROUGH", fields=["geometry", "type", "AREA_KM2"]), 
    )
    .transform_filter(month_selection & weather_selection & vehicle_selection)
    .transform_aggregate(sumCollisions = "sum(VALID)", groupby = ["BOROUGH", "AREA_KM2", "geometry", "type"])
    .transform_calculate(COLLISIONS_KM2 = "datum.sumCollisions / datum.AREA_KM2")
    .encode(
        color=alt.condition(ny_map_selection, alt.Color("COLLISIONS_KM2:Q", scale=alt.Scale(scheme='greens'), legend=alt.Legend(title="Collisions per km2")), alt.value("lightgray")),
        tooltip=[alt.Tooltip("BOROUGH:N", title="Borough"), alt.Tooltip("COLLISIONS_KM2:Q", title="Collisions per km2")],
    ).properties(
        width=300,
        height=300,
        title="NYC Community Districts"
    ).add_params(
        ny_map_selection
    )
)    

In [ ]:
((months | vehicles | weather) & (ny_map)).resolve_legend(color="independent")

Weekdays

In [ ]:
weekdayorder = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]


day_selection = alt.selection_interval(encodings=["x"], empty=True)

weekdays_df = collisions.groupby(["CRASH DAY", "CRASH WEEKDAY", "CRASH WEEK NUMBER", "MONTH", "VEHICLE", "WEATHER", "DAY", "BOROUGH"]).agg({"VALID": "sum"}).reset_index()
weekdays = (
    alt.Chart(collisions)
    .mark_rect()
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & ny_map_selection
    )
    .encode(
        x=alt.X("CRASH WEEKDAY:O", title="Crash Weekday", sort=weekdayorder, axis=alt.Axis(labelAngle=0)),
        y=alt.Y("CRASH WEEK NUMBER:O", title="Crash Week Number"),
        color=alt.Color("sum(VALID):Q", scale=alt.Scale(scheme="greens"), title="Collisions"),
        opacity=alt.condition(day_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("CRASH DAY:O", title="Crash day"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    )
    .add_params(
        day_selection
    ).properties(
        width=300,
        height=300,
    )
)

weekdays += (
    alt.Chart(collisions)
    .mark_text(align="left", size=10)
    .transform_filter(month_selection)
    .transform_aggregate(groupby = ["DAY", "MONTH", "CRASH WEEKDAY", "CRASH WEEK NUMBER"])
    .encode(
        x=alt.X("CRASH WEEKDAY:O", title=None, sort=weekdayorder),
        y=alt.Y("CRASH WEEK NUMBER:O", title=None),
        text=alt.Text("DAY:N") 
    )
)

In [ ]:
((months | vehicles | weather) & (ny_map | weekdays).resolve_legend(color="independent").resolve_scale(color="independent"))

Line chart

In [ ]:
hour_selection = alt.selection_point(fields=["HOUR"], on="mouseover", nearest=True, empty=False)

hours_df = collisions.groupby(["CRASH DAY", "CRASH WEEKDAY", "MONTH", "VEHICLE", "WEATHER", "BOROUGH", "DAY", "HOUR"]).agg({"VALID": "sum"}).reset_index()

hours = (
    alt.Chart(hours_df)
    .mark_line(color=colors[primary])
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(title="Hours", labelAngle=0)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
        color=alt.Color("BOROUGH:N", legend=alt.Legend(title="Borough")),
        tooltip=[alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        ny_map_selection
    ).properties(width=700, height=300)
)

hours_tooltip = (
    alt.Chart(hours_df)
    .mark_rule(opacity=0)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis = alt.Axis(title=None)),
        opacity=alt.condition(hour_selection, alt.value(1), alt.value(0)),
    )
)

hours_tooltip += (
    alt.Chart(hours_df)
    .mark_circle(opacity=0, size=50)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(title=None, labelAngle=0)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title=None)),
        color=alt.Color("BOROUGH:N"),
        opacity=alt.condition(hour_selection, alt.value(1), alt.value(0)),
    )
)

hours_tooltip += (
    alt.Chart(hours_df)
    .mark_text(align='left', dx=5, dy=-5, size=10)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(title=None, labelAngle=0)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title=None)),
        text=alt.condition(hour_selection, "sum(VALID):Q", alt.value(' ')),
        color=alt.Color("BOROUGH:N")
    )
    .add_params(
        hour_selection
    )
)

In [ ]:
(
    (months | vehicles | weather) &
    (ny_map | weekdays).resolve_legend(color="independent").resolve_scale(color="independent") &
    (hours + hours_tooltip)
).resolve_legend(color="independent")